In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat, quietly = T)
        library(openxlsx, quietly = T)
        library(ggpubr, quietly = T)
        library(plyr, quietly = T)
        library(dplyr, quietly = T)

    })
})

data_path = '/data3/hratch/norcross_abc/'

In [2]:
abc.integrated<-readRDS(paste0(data_path, 'processed/abc_annotated.RDS'))

Specifiy the cell types and context comparisons to test for:

In [4]:
unique(abc.integrated$orig.ident)

[1] UNTR     ABC      DT_Veh   DT_ABC   aCD4_ABC
Levels: UNTR ABC DT_Veh DT_ABC aCD4_ABC

In [5]:
cell.types<-c('CD8+ TN/EA-ISG')
comparisons<-list(dt.effect = c('DT_ABC', 'ABC')) # second entry is baseline

Since we are testing differences in the same cell type across contexts, we employ DE tests that can control for technical effects. Latent variables that account for technical effects have been [shown](https://www.biorxiv.org/content/10.1101/2022.03.15.484475v1) to be effective for DE across contexts. We will use MAST and the CDR (cellular detection rate) which has been [shown](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-015-0844-5) to be an effective latent variable for technical effects

# CDR

First, we calculate the CDR from the LogNormalized expression matrix:



In [10]:
freq<-function(expr){
    nonzero.counts<-rowSums(expr !=0 ) # get # of nonzero cells per gene
    return(nonzero.counts/dim(expr)[[2]])
}

expr = abc.integrated@assays$RNA@data # log-normalized matrix
expr<-expr[which(freq(expr)>0),] # remove invariant genes

In [11]:
thresh = 0 # calculate CDR on non-zero frequency (NOTE: code will need to be changed if setting higher thresh)
if (thresh == 0){
    cdr<-unlist(unname(scale(colSums(expr!=thresh))[, 1])) # calculate CDR as in MAST tutorial (https://www.bioconductor.org/packages/release/bioc/vignettes/MAST/inst/doc/MAITAnalysis.html)
    cdr.2<-unlist(unname(colSums(expr > thresh)/dim(expr)[[1]])) # calculate as in MAST manuscript
}else{
    stop('Need to implement this if using')
}

Note, although the two methods to calculate the CDR give different absolute values, they have perfect correlation (we will proceed with the tutorial recommended CDR calculation):

In [12]:
identical(cdr, cdr.2)
cor(cdr, cdr.2,  method = "spearman", use = "complete.obs")

[1] FALSE

[1] 1

In [13]:
abc.integrated@meta.data[['cellular.detection.rate']]<-cdr # add cdr to object

# RUN MAST DE

In [15]:
MAST.de<-function(Cell.Type.Level2, context.treat, context.base, latent.vars, min.pct, lfc.thresh){
    abc.subset<-subset(x = abc.integrated, subset = Cell.Type.Level2 == ct)
    Idents(abc.subset)<-'orig.ident'
    
    suppressWarnings({
        suppressMessages({
            de.res<-FindMarkers(object = abc.subset, 
                                ident.1 = context.treat, ident.2 = context.base,
                                assay = 'RNA', only.pos = F, 
                                slot = 'data', test.use = 'MAST', 
                                latent.vars = latent.vars,
                                min.pct = min.pct, 
                                logfc.threshold = lfc.thresh 
                                              )
            })
    })
    
    names(de.res)[names(de.res) == 'p_val_adj'] <- 'bonferroni.adjusted' # rename to specify correction type
    # get the B-H to be less stringent than the native Seurat Bonferroni
    de.res[['BH.adjusted']]<-p.adjust(p = de.res$p_val, method = "BH") 
    de.res[['gene']]<-rownames(de.res)
    de.res[['Cell.Type.Level2']]<-ct
    de.res[['Comparison']]<-paste0(context.treat, '_vs_', context.base)
    
    return(de.res)
}

In [18]:
MAST.de.res<-list()
for (comparison in comparisons){
    for (ct in cell.types){
        context.treat<-comparison[[1]]
        context.base<-comparison[[2]]
        cond.name<-paste0(ct, '_', paste0(comparison, collapse = 'vs'))

        MAST.de.res[[cond.name]]<-MAST.de(cell.type, context.treat, context.base, 
                                      latent.vars = 'cellular.detection.rate', 
                                         min.pct = 0.1, lfc.thresh = 0.5)
    }
}


In [24]:
de.res<-do.call("rbind", MAST.de.res)
de.res<-de.res[de.res$BH.adjusted <= 0.1,]

print('# of DE genes after filtering:')
table(de.res$Cell.Type, de.res$Comparison)

[1] "# of DE genes after filtering:"


                
                 DT_ABC_vs_ABC
  CD8+ TN/EA-ISG           225

In [25]:
write.csv(de.res, paste0(data_path, 'processed/', 'ISG_de.csv'))